# Network Expansion and Topology Modification

Demonstrates how to programmatically modify system topology by tapping existing lines and splitting buses. This is essential for planning studies where new substations or interconnections are evaluated.

In [ ]:
from esapp import GridWorkBench, Branch, Bus

wb = GridWorkBench(r"case.pwb")

'open' took: 3.2314 sec


Select a specific branch to modify. We'll use the 11th branch in the list for this example.

In [4]:
branches = wb[Branch]

b = branches.iloc[10]
tobus = b['BusNum']
frombus = b['BusNum:1']
circuit = b['LineCircuit']
branch_str = f"[BRANCH {tobus} {frombus} {circuit}]"

Then, we tap an existing transmission line

In [5]:

new_bus_num = wb[Bus, 'BusNum'].max() + 100

wb.esa.TapTransmissionLine(
    branch_str, 
    50.0, 
    new_bus_num,
    "CAPACITOR", 
    False, False, 
    "Tapped_Substation"
)

print(f"Tapped line at 50% to create Bus {new_bus_num}")


Tapped line at 50% to create Bus 137


Split a bus

In [7]:
target_bus = 1
split_bus_num = wb[Bus, 'BusNum'].max() + 1

wb.esa.SplitBus(f"[BUS {target_bus}]", split_bus_num)

print(f"Split Bus {target_bus} to create Bus {split_bus_num}")

Split Bus 1 to create Bus 138


Then we validate changes with power flow

In [8]:
wb.pflow()

0     0.993355-0.019419j
1     0.988897-0.067891j
2     0.981193-0.081206j
3     0.973882-0.097994j
4     0.988339-0.035719j
5     0.976643-0.094524j
6     0.975783-0.096903j
7     0.973776-0.097237j
8     0.985792-0.043874j
9     0.976172-0.096608j
10    0.968600-0.110751j
11    0.971614-0.105523j
12    0.973420-0.103939j
13    0.982291-0.053822j
14    0.973217-0.102800j
15    0.975958-0.110438j
16    0.968262-0.118577j
17    0.968432-0.120988j
18    0.989440-0.060763j
19    0.989079-0.033916j
20    0.975803-0.089976j
21    0.996289-0.014687j
22    0.999325-0.036749j
23    0.991324-0.064225j
24    0.996572-0.009197j
25    0.993144-0.062429j
26    1.003133+0.023863j
27    0.999992+0.004085j
28    0.995382-0.001690j
29    0.995771-0.007340j
30    0.994545-0.036499j
31    0.990953-0.022854j
32    0.994241-0.052982j
33    0.999338-0.036369j
34    0.999978+0.006569j
35    0.994644-0.061972j
36    0.999990+0.004388j
37    0.981414-0.082958j
38    0.993355-0.019419j
dtype: complex128